In [1]:
######################################################
# Juypter
######################################################
%load_ext autoreload
%autoreload
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))
display(HTML("""<style>div.output_area{max-height:10000px;overflow:scroll;}</style>"""))



################################################################################
## Music DB
################################################################################
from mainDB import mainDB
from musicDBMap import musicDBMap
from masterDBMatchClass import masterDBMatchClass
from matchDBArtist import matchDBArtist


######################################################
# Versions
######################################################
import sys
print("Python: {0}".format(sys.version))
import datetime as dt
start = dt.datetime.now()
print("Notebook Last Run Initiated: "+str(start))

/Users/tgadfort/opt/anaconda3/envs/py37/lib/python3.7/site-packages/sklearn/externals/joblib/__init__.py:15: FutureWarning: sklearn.externals.joblib is deprecated in 0.21 and will be removed in 0.23. Please import this functionality directly from joblib, which can be installed with: pip install joblib. If this warning is raised when loading pickled models, you may need to re-serialize those models with scikit-learn 0.21+.
  warnings.warn(msg, category=FutureWarning)


Python: 3.7.7 (default, Mar 26 2020, 10:32:53) 
[Clang 4.0.1 (tags/RELEASE_401/final)]
Notebook Last Run Initiated: 2021-02-06 20:42:46.312628


# AllMusic

In [ ]:
%load_ext autoreload
%autoreload
from dbArtistsAllMusic import dbArtistsAllMusic
from dbArtistsParse import dbArtistsPrimary, dbArtistsCredit, dbArtistsExtra
from dbArtistsParse import dbArtistsAssertCredit

for modVal in range(100):
    dbAP = dbArtistsPrimary(dbArtistsAllMusic())
    dometa = dbAP.parse(modVal, previousDays=1, force=False)
    if dometa:
        dbAP.createArtistMetadata(modVal)
        dbAP.createAlbumMetadata(modVal)

In [ ]:
# This downloads missing credit files
for modVal in range(100):
    try:
        dbac = dbArtistsAssertCredit(dbArtistsAllMusic())
        dbac.parseModVal(modVal)
        dbac.downloadUnknownArtistCredits()
    except:
        continue

In [ ]:
for modVal in range(100):
    dbAP = dbArtistsCredit(dbArtistsAllMusic())
    dbAP.parse(modVal, previousDays=200)

In [ ]:
for modVal in range(100):
    dbAP = dbArtistsPrimary(dbArtistsAllMusic())
    dbAP.createArtistMetadata(modVal)
    dbAP.createAlbumMetadata(modVal)

# MusicBrainz

In [ ]:
%load_ext autoreload
%autoreload
from dbArtistsMusicBrainz import dbArtistsMusicBrainz
from dbArtistsParse import dbArtistsPrimary, dbArtistsCredit, dbArtistsExtra
from dbArtistsParse import dbArtistsAssertExtra

In [ ]:
for modVal in range(100):
    dbAP = dbArtistsPrimary(dbArtistsMusicBrainz())
    dometa = dbAP.parse(modVal, previousDays=1, force=False)
    if dometa:
        dbAP.createArtistMetadata(modVal)
        dbAP.createAlbumMetadata(modVal)

#for modVal in range(100):
#    dbAP = dbArtistsExtra(dbArtistsMusicBrainz())
#    dbAP.parse(modVal)

In [ ]:
for modVal in range(100):
    dbAP = dbArtistsAssertExtra(dbArtistsMusicBrainz())
    dbAP.parseModVal(modVal)
    dbAP.downloadMissingArtistExtras()

# Discogs

In [ ]:
%load_ext autoreload
%autoreload
from dbArtistsDiscogs import dbArtistsDiscogs
from dbArtistsParse import dbArtistsPrimary, dbArtistsCredit, dbArtistsExtra
from dbArtistsParse import dbArtistsAssertExtra, dbArtistsUnofficial, dbArtistsAssertUnofficial

In [ ]:
for modVal in range(100):
    dbAP = dbArtistsPrimary(dbArtistsDiscogs())
    dometa = dbAP.parse(modVal, previousDays=2, force=False)
    if dometa:
        dbAP.createArtistMetadata(modVal)
        dbAP.createAlbumMetadata(modVal)

if False:
    for modVal in range(100):
        break
        dbAP = dbArtistsExtra(dbArtistsDiscogs())
        dbAP.parse(modVal, previousDays=10, force=True)

    for modVal in range(100):
        break
        dbAP = dbArtistsUnofficial(dbArtistsDiscogs())
        dbAP.parse(modVal, previousDays=10, force=True)

    for modVal in range(100):
        dbAP = dbArtistsPrimary(dbArtistsDiscogs())

In [ ]:
for modVal in range(100):
    dbAP = dbArtistsAssertUnofficial(dbArtistsDiscogs())
    dbAP.parseModVal(modVal)
    dbAP.downloadMissingArtistUnofficials()

# RateYourMusic

In [7]:
#def getAllRawHTMLFiles(self, dirVal):
#files = findExt(setDir(dirVal, "data"), ext=".html")
#return files
from searchUtils import findExt
files = findExt("/Volumes/Piggy/Discog/artists-rateyourmusic/data/", '.html')
len(files)

871

In [6]:
%load_ext autoreload
%autoreload
from dbArtistsRateYourMusic import dbArtistsRateYourMusic
from dbArtistsParse import dbArtistsPrimary, dbArtistsCredit, dbArtistsExtra
from dbArtistsParse import dbArtistsAssertCredit, dbArtistsRawHTML

dbAP = dbArtistsRawHTML(dbArtistsRateYourMusic())
dbAP.parse(previousDays=20000000)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
=========================   rateyourmusic   =========================
  ===> Found new 871 files (< 20000000 days) to parse
Parsing /Volumes/Piggy/Discog/artists-rateyourmusic/data/Alejandro Sanz Albums: songs, discography, biography, and listening guide - Rate Your Music.html
  --> This file is 37.3kB.
Parsing /Volumes/Piggy/Discog/artists-rateyourmusic/data/Bastille Albums: songs, discography, biography, and listening guide - Rate Your Music.html
  --> This file is 36.2kB.
Parsing /Volumes/Piggy/Discog/artists-rateyourmusic/data/Bob Seger Albums: songs, discography, biography, and listening guide - Rate Your Music.html
  --> This file is 41.0kB.
Parsing /Volumes/Piggy/Discog/artists-rateyourmusic/data/Bon Jovi Albums: songs, discography, biography, and listening guide - Rate Your Music.html
  --> This file is 43.1kB.
Parsing /Volumes/Piggy/Discog/artists-rateyourmusic/data/Cosmic Gate Albums: song

  --> This file is 36.3kB.
Parsing /Volumes/Piggy/Discog/artists-rateyourmusic/data/Whigfield Albums: songs, discography, biography, and listening guide - Rate Your Music.html
  --> This file is 31.1kB.
Parsing /Volumes/Piggy/Discog/artists-rateyourmusic/data/Yellowman Albums: songs, discography, biography, and listening guide - Rate Your Music.html
  --> This file is 37.7kB.
Parsing /Volumes/Piggy/Discog/artists-rateyourmusic/data/Young Fame Albums: songs, discography, biography, and listening guide - Rate Your Music.html
  --> This file is 17.1kB.
Parsing /Volumes/Piggy/Discog/artists-rateyourmusic/data/Øresund Space Collective Albums: songs, discography, biography, and listening guide - Rate Your Musi.html
  --> This file is 32.9kB.
Parsing /Volumes/Piggy/Discog/artists-rateyourmusic/data/Kali Raps Albums: songs, discography, biography, and listening guide - Rate Your Music.html
  --> This file is 17.0kB.
Parsing /Volumes/Piggy/Discog/artists-rateyourmusic/data/Freck Billionaire Alb

  --> This file is 43.6kB.
Parsing /Volumes/Piggy/Discog/artists-rateyourmusic/data/Alison Krauss and Union Station Albums: songs, discography, biography, and listening guide - Rate Yo.html
  --> This file is 21.9kB.
Parsing /Volumes/Piggy/Discog/artists-rateyourmusic/data/Alison Moyet Albums: songs, discography, biography, and listening guide - Rate Your Music.html
  --> This file is 35.8kB.
Parsing /Volumes/Piggy/Discog/artists-rateyourmusic/data/Aloe Blacc Albums: songs, discography, biography, and listening guide - Rate Your Music.html
  --> This file is 33.5kB.
Parsing /Volumes/Piggy/Discog/artists-rateyourmusic/data/Alphaville Albums: songs, discography, biography, and listening guide - Rate Your Music.html
  --> This file is 36.2kB.
Parsing /Volumes/Piggy/Discog/artists-rateyourmusic/data/America Albums: songs, discography, biography, and listening guide - Rate Your Music.html
  --> This file is 41.1kB.
Parsing /Volumes/Piggy/Discog/artists-rateyourmusic/data/Amy Rigby Albums: s

  --> This file is 28.9kB.
Parsing /Volumes/Piggy/Discog/artists-rateyourmusic/data/Gipsy Kings Albums: songs, discography, biography, and listening guide - Rate Your Music.html
  --> This file is 34.2kB.
Parsing /Volumes/Piggy/Discog/artists-rateyourmusic/data/Glimmer Albums: songs, discography, biography, and listening guide - Rate Your Music.html
  --> This file is 15.2kB.
Parsing /Volumes/Piggy/Discog/artists-rateyourmusic/data/HALO Albums: songs, discography, biography, and listening guide - Rate Your Music.html
  --> This file is 21.6kB.
Parsing /Volumes/Piggy/Discog/artists-rateyourmusic/data/Intro To India Albums: songs, discography, biography, and listening guide - Rate Your Music.html
  --> This file is 14.7kB.
Parsing /Volumes/Piggy/Discog/artists-rateyourmusic/data/Jackson 5 Albums: songs, discography, biography, and listening guide - Rate Your Music.html
  --> This file is 41.6kB.
Parsing /Volumes/Piggy/Discog/artists-rateyourmusic/data/Jackson Browne Albums: songs, discog

  --> This file is 32.5kB.
Parsing /Volumes/Piggy/Discog/artists-rateyourmusic/data/The Bangles Albums: songs, discography, biography, and listening guide - Rate Your Music.html
  --> This file is 41.5kB.
Parsing /Volumes/Piggy/Discog/artists-rateyourmusic/data/The Beatniks Albums: songs, discography, biography, and listening guide - Rate Your Music.html
  --> This file is 20.2kB.
Parsing /Volumes/Piggy/Discog/artists-rateyourmusic/data/The Fisticuffs Albums: songs, discography, biography, and listening guide - Rate Your Music.html
  --> This file is 17.8kB.
Parsing /Volumes/Piggy/Discog/artists-rateyourmusic/data/The Future Sound of London Albums: songs, discography, biography, and listening guide - Rate Your Mu.html
  --> This file is 40.5kB.
Parsing /Volumes/Piggy/Discog/artists-rateyourmusic/data/The Glimmer Albums: songs, discography, biography, and listening guide - Rate Your Music.html
  --> This file is 15.6kB.
Parsing /Volumes/Piggy/Discog/artists-rateyourmusic/data/The Guess 

...

  --> This file is 42.1kB.
Parsing /Volumes/Piggy/Discog/artists-rateyourmusic/data/Stone Temple Pilots Albums: songs, discography, biography, and listening guide - Rate Your Music.html
  --> This file is 41.0kB.
Parsing /Volumes/Piggy/Discog/artists-rateyourmusic/data/Wheat Albums: songs, discography, biography, and listening guide - Rate Your Music.html
  --> This file is 24.0kB.
Parsing /Volumes/Piggy/Discog/artists-rateyourmusic/data/Wheatus Albums: songs, discography, biography, and listening guide - Rate Your Music.html
  --> This file is 25.1kB.
Parsing /Volumes/Piggy/Discog/artists-rateyourmusic/data/Lorenz Hart Albums: songs, discography, biography, and listening guide - Rate Your Music.html
  --> This file is 15.7kB.
Parsing /Volumes/Piggy/Discog/artists-rateyourmusic/data/Bob Ludwig Albums: songs, discography, biography, and listening guide - Rate Your Music.html
  --> This file is 14.4kB.
Parsing /Volumes/Piggy/Discog/artists-rateyourmusic/data/Keith Anderson Albums: songs,

  --> This file is 28.0kB.
Parsing /Volumes/Piggy/Discog/artists-rateyourmusic/data/Seán Óg Albums: songs, discography, biography, and listening guide - Rate Your Music.html
  --> This file is 18.9kB.
Parsing /Volumes/Piggy/Discog/artists-rateyourmusic/data/Fang Island Albums: songs, discography, biography, and listening guide - Rate Your Music.html
  --> This file is 23.4kB.
Parsing /Volumes/Piggy/Discog/artists-rateyourmusic/data/Die Singeborgs Albums: songs, discography, biography, and listening guide - Rate Your Music.html
  --> This file is 18.2kB.
Parsing /Volumes/Piggy/Discog/artists-rateyourmusic/data/The Monsters Albums: songs, discography, biography, and listening guide - Rate Your Music.html
  --> This file is 23.1kB.
Parsing /Volumes/Piggy/Discog/artists-rateyourmusic/data/La Lupe Albums: songs, discography, biography, and listening guide - Rate Your Music.html
  --> This file is 35.5kB.
Parsing /Volumes/Piggy/Discog/artists-rateyourmusic/data/6ix9ine Albums: songs, disco

  --> This file is 16.3kB.
Parsing /Volumes/Piggy/Discog/artists-rateyourmusic/data/Petter Ferraz Albums: songs, discography, biography, and listening guide - Rate Your Music.html
  --> This file is 19.0kB.
Parsing /Volumes/Piggy/Discog/artists-rateyourmusic/data/The Fireflies Albums: songs, discography, biography, and listening guide - Rate Your Music.html
  --> This file is 23.0kB.
Parsing /Volumes/Piggy/Discog/artists-rateyourmusic/data/Lee and The Sounds Albums: songs, discography, biography, and listening guide - Rate Your Music.html
  --> This file is 18.0kB.
Parsing /Volumes/Piggy/Discog/artists-rateyourmusic/data/The Kanyu Tree Albums: songs, discography, biography, and listening guide - Rate Your Music.html
  --> This file is 18.6kB.
Parsing /Volumes/Piggy/Discog/artists-rateyourmusic/data/Scuba Albums: songs, discography, biography, and listening guide - Rate Your Music.html
  --> This file is 35.3kB.
Parsing /Volumes/Piggy/Discog/artists-rateyourmusic/data/Scuba Death Albums

  --> This file is 32.7kB.
Parsing /Volumes/Piggy/Discog/artists-rateyourmusic/data/Vint Albums: songs, discography, biography, and listening guide - Rate Your Music.html
  --> This file is 22.1kB.
Parsing /Volumes/Piggy/Discog/artists-rateyourmusic/data/India Martínez Albums: songs, discography, biography, and listening guide - Rate Your Music.html
  --> This file is 24.7kB.
Parsing /Volumes/Piggy/Discog/artists-rateyourmusic/data/J Soul Brothers Albums: songs, discography, biography, and listening guide - Rate Your Music.html
  --> This file is 23.8kB.
Parsing /Volumes/Piggy/Discog/artists-rateyourmusic/data/スピッツ Albums: songs, discography, biography, and listening guide - Rate Your Music.html
  --> This file is 34.9kB.
Parsing /Volumes/Piggy/Discog/artists-rateyourmusic/data/登坂広臣 Albums: songs, discography, biography, and listening guide - Rate Your Music.html
  --> This file is 18.2kB.
Parsing /Volumes/Piggy/Discog/artists-rateyourmusic/data/ちゃんみな Albums: songs, discography, biog

  --> This file is 18.8kB.
Parsing /Volumes/Piggy/Discog/artists-rateyourmusic/data/FY Albums: songs, discography, biography, and listening guide - Rate Your Music.html
  --> This file is 18.9kB.
Parsing /Volumes/Piggy/Discog/artists-rateyourmusic/data/Snik Albums: songs, discography, biography, and listening guide - Rate Your Music.html
  --> This file is 17.7kB.
Parsing /Volumes/Piggy/Discog/artists-rateyourmusic/data/עדן בן זקן Albums: songs, discography, biography, and listening guide - Rate Your Music.html
  --> This file is 18.7kB.
Parsing /Volumes/Piggy/Discog/artists-rateyourmusic/data/Pop Levi Albums: songs, discography, biography, and listening guide - Rate Your Music.html
  --> This file is 26.6kB.
Parsing /Volumes/Piggy/Discog/artists-rateyourmusic/data/עדן חסון Albums: songs, discography, biography, and listening guide - Rate Your Music.html
  --> This file is 17.8kB.
Parsing /Volumes/Piggy/Discog/artists-rateyourmusic/data/Ishai Adar Albums: songs, discography, biography,

In [ ]:
for modVal in range(100):
    dbAP = dbArtistsPrimary(dbArtistsRateYourMusic())
    dometa = dbAP.parse(modVal, previousDays=1, force=False)
    if dometa:
        dbAP.createArtistMetadata(modVal)
        dbAP.createAlbumMetadata(modVal)

# Deezer

In [2]:
%load_ext autoreload
%autoreload
from dbArtistsDeezer import dbArtistsDeezer
from dbArtistsParse import dbArtistsPrimary, dbArtistsCredit, dbArtistsExtra
from dbArtistsParse import dbArtistsAssertExtra, dbArtistsUnofficial, dbArtistsAssertUnofficial

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [3]:
for modVal in range(100):
    dbAP = dbArtistsPrimary(dbArtistsDeezer())
    dometa = dbAP.parse(modVal, previousDays=1, force=True)
    if dometa:
        dbAP.createArtistMetadata(modVal)
        dbAP.createAlbumMetadata(modVal)

=========================   deezer   =========================
  ===> Parsing all 90 files for modval 0
  ===> Forcing Reloads of ModVal=0
	Processed 0/90 files.
No track table
No track table
No track table
No track table
No track table
No track table
No track table
No track table
No track table
No track table
No track table
No track table
No track table
No track table
No track table
No track table
Saving 90 new artist IDs to /Users/tgadfort/Music/Discog/artists-deezer-db/0-DB.p
Saving 90 total artist IDs to /Users/tgadfort/Music/Discog/artists-deezer-db/0-DB.p
Saving 1758 total artist media
  --> This file is 80.0kB.
  ===> Found 90 previous data for ModVal=0
Saving 90 new artist IDs name data to /Users/tgadfort/Music/Discog/artists-deezer-db/metadata/0-Metadata.p
  --> This file is 3.3kB.
  ===> Found 90 previous data for ModVal=0
Saving 90 new artist IDs name data to /Users/tgadfort/Music/Discog/artists-deezer-db/metadata/0-MediaMetadata.p
  --> This file is 4.7kB.
=================

  ===> Found 98 previous data for ModVal=6
Saving 98 new artist IDs name data to /Users/tgadfort/Music/Discog/artists-deezer-db/metadata/6-Metadata.p
  --> This file is 3.6kB.
  ===> Found 98 previous data for ModVal=6
Saving 98 new artist IDs name data to /Users/tgadfort/Music/Discog/artists-deezer-db/metadata/6-MediaMetadata.p
  --> This file is 5.2kB.
=========================   deezer   =========================
  ===> Parsing all 114 files for modval 7
  ===> Forcing Reloads of ModVal=7
	Processed 0/114 files.
No track table
No track table
No track table
No track table
No track table
No track table
No track table
No track table
No track table
No track table
No track table
No track table
No track table
No track table
No track table
No track table
No track table
No track table
No track table
No track table
No track table
	Processed 100/114 files.
No track table
No track table
No track table
Saving 114 new artist IDs to /Users/tgadfort/Music/Discog/artists-deezer-db/7-DB.p
Saving 114

No track table
No track table
No track table
No track table
No track table
No track table
No track table
No track table
No track table
No track table
No track table
No track table
No track table
No track table
No track table
No track table
No track table
No track table
No track table
No track table
Saving 86 new artist IDs to /Users/tgadfort/Music/Discog/artists-deezer-db/13-DB.p
Saving 86 total artist IDs to /Users/tgadfort/Music/Discog/artists-deezer-db/13-DB.p
Saving 1597 total artist media
  --> This file is 73.6kB.
  ===> Found 86 previous data for ModVal=13
Saving 86 new artist IDs name data to /Users/tgadfort/Music/Discog/artists-deezer-db/metadata/13-Metadata.p
  --> This file is 3.0kB.
  ===> Found 86 previous data for ModVal=13
Saving 86 new artist IDs name data to /Users/tgadfort/Music/Discog/artists-deezer-db/metadata/13-MediaMetadata.p
  --> This file is 4.3kB.
=========================   deezer   =========================
  ===> Parsing all 99 files for modval 14
  ===> F

  --> This file is 4.8kB.
=========================   deezer   =========================
  ===> Parsing all 137 files for modval 21
  ===> Forcing Reloads of ModVal=21
	Processed 0/137 files.
No track table
No track table
No track table
No track table
No track table
No track table
No track table
No track table
No track table
No track table
No track table
No track table
No track table
No track table
No track table
No track table
No track table
No track table
	Processed 100/137 files.
No track table
No track table
No track table
No track table
No track table
No track table
No track table
No track table
No track table
No track table
No track table
Saving 137 new artist IDs to /Users/tgadfort/Music/Discog/artists-deezer-db/21-DB.p
Saving 137 total artist IDs to /Users/tgadfort/Music/Discog/artists-deezer-db/21-DB.p
Saving 2576 total artist media
  --> This file is 119.3kB.
  ===> Found 137 previous data for ModVal=21
Saving 137 new artist IDs name data to /Users/tgadfort/Music/Discog/artis

No track table
No track table
No track table
No track table
No track table
No track table
No track table
No track table
No track table
No track table
No track table
No track table
No track table
No track table
No track table
No track table
Saving 82 new artist IDs to /Users/tgadfort/Music/Discog/artists-deezer-db/27-DB.p
Saving 82 total artist IDs to /Users/tgadfort/Music/Discog/artists-deezer-db/27-DB.p
Saving 1465 total artist media
  --> This file is 66.4kB.
  ===> Found 82 previous data for ModVal=27
Saving 82 new artist IDs name data to /Users/tgadfort/Music/Discog/artists-deezer-db/metadata/27-Metadata.p
  --> This file is 3.0kB.
  ===> Found 82 previous data for ModVal=27
Saving 82 new artist IDs name data to /Users/tgadfort/Music/Discog/artists-deezer-db/metadata/27-MediaMetadata.p
  --> This file is 4.2kB.
=========================   deezer   =========================
  ===> Parsing all 102 files for modval 28
  ===> Forcing Reloads of ModVal=28
	Processed 0/102 files.
No trac

...

No track table
No track table
No track table
No track table
No track table
No track table
No track table
No track table
No track table
No track table
No track table
No track table
No track table
No track table
No track table
No track table
No track table
No track table
No track table
No track table
Saving 84 new artist IDs to /Users/tgadfort/Music/Discog/artists-deezer-db/70-DB.p
Saving 84 total artist IDs to /Users/tgadfort/Music/Discog/artists-deezer-db/70-DB.p
Saving 1615 total artist media
  --> This file is 72.5kB.
  ===> Found 84 previous data for ModVal=70
Saving 84 new artist IDs name data to /Users/tgadfort/Music/Discog/artists-deezer-db/metadata/70-Metadata.p
  --> This file is 3.0kB.
  ===> Found 84 previous data for ModVal=70
Saving 84 new artist IDs name data to /Users/tgadfort/Music/Discog/artists-deezer-db/metadata/70-MediaMetadata.p
  --> This file is 4.2kB.
=========================   deezer   =========================
  ===> Parsing all 126 files for modval 71
  ===> 

No track table
No track table
No track table
No track table
No track table
No track table
No track table
Saving 99 new artist IDs to /Users/tgadfort/Music/Discog/artists-deezer-db/76-DB.p
Saving 99 total artist IDs to /Users/tgadfort/Music/Discog/artists-deezer-db/76-DB.p
Saving 1769 total artist media
  --> This file is 84.6kB.
  ===> Found 99 previous data for ModVal=76
Saving 99 new artist IDs name data to /Users/tgadfort/Music/Discog/artists-deezer-db/metadata/76-Metadata.p
  --> This file is 3.6kB.
  ===> Found 99 previous data for ModVal=76
Saving 99 new artist IDs name data to /Users/tgadfort/Music/Discog/artists-deezer-db/metadata/76-MediaMetadata.p
  --> This file is 4.9kB.
=========================   deezer   =========================
  ===> Parsing all 96 files for modval 77
  ===> Forcing Reloads of ModVal=77
	Processed 0/96 files.
No track table
No track table
No track table
No track table
No track table
No track table
No track table
No track table
No track table
No track 

No track table
No track table
No track table
No track table
No track table
No track table
No track table
Saving 285 new artist IDs to /Users/tgadfort/Music/Discog/artists-deezer-db/82-DB.p
Saving 285 total artist IDs to /Users/tgadfort/Music/Discog/artists-deezer-db/82-DB.p
Saving 2487 total artist media
  --> This file is 138.1kB.
  ===> Found 285 previous data for ModVal=82
Saving 285 new artist IDs name data to /Users/tgadfort/Music/Discog/artists-deezer-db/metadata/82-Metadata.p
  --> This file is 11.2kB.
  ===> Found 285 previous data for ModVal=82
Saving 285 new artist IDs name data to /Users/tgadfort/Music/Discog/artists-deezer-db/metadata/82-MediaMetadata.p
  --> This file is 11.9kB.
=========================   deezer   =========================
  ===> Parsing all 80 files for modval 83
  ===> Forcing Reloads of ModVal=83
	Processed 0/80 files.
No track table
No track table
No track table
No track table
No track table
No track table
No track table
No track table
No track table


Saving 80 new artist IDs to /Users/tgadfort/Music/Discog/artists-deezer-db/90-DB.p
Saving 80 total artist IDs to /Users/tgadfort/Music/Discog/artists-deezer-db/90-DB.p
Saving 1448 total artist media
  --> This file is 66.5kB.
  ===> Found 80 previous data for ModVal=90
Saving 80 new artist IDs name data to /Users/tgadfort/Music/Discog/artists-deezer-db/metadata/90-Metadata.p
  --> This file is 2.9kB.
  ===> Found 80 previous data for ModVal=90
Saving 80 new artist IDs name data to /Users/tgadfort/Music/Discog/artists-deezer-db/metadata/90-MediaMetadata.p
  --> This file is 4.1kB.
=========================   deezer   =========================
  ===> Parsing all 132 files for modval 91
  ===> Forcing Reloads of ModVal=91
	Processed 0/132 files.
No track table
No track table
No track table
No track table
No track table
No track table
No track table
No track table
No track table
No track table
No track table
No track table
No track table
No track table
No track table
No track table
No trac

No track table
No track table
No track table
No track table
No track table
No track table
No track table
No track table
No track table
No track table
No track table
No track table
No track table
No track table
No track table
No track table
No track table
No track table
No track table
No track table
No track table
	Processed 100/103 files.
Saving 103 new artist IDs to /Users/tgadfort/Music/Discog/artists-deezer-db/97-DB.p
Saving 103 total artist IDs to /Users/tgadfort/Music/Discog/artists-deezer-db/97-DB.p
Saving 1661 total artist media
  --> This file is 79.2kB.
  ===> Found 103 previous data for ModVal=97
Saving 103 new artist IDs name data to /Users/tgadfort/Music/Discog/artists-deezer-db/metadata/97-Metadata.p
  --> This file is 3.8kB.
  ===> Found 103 previous data for ModVal=97
Saving 103 new artist IDs name data to /Users/tgadfort/Music/Discog/artists-deezer-db/metadata/97-MediaMetadata.p
  --> This file is 5.3kB.
=========================   deezer   =========================
  =

# LastFM

In [ ]:
%load_ext autoreload
%autoreload
from dbArtistsLastFM import dbArtistsLastFM
from dbArtistsParse import dbArtistsPrimary, dbArtistsCredit, dbArtistsExtra
from dbArtistsParse import dbArtistsAssertExtra, dbArtistsUnofficial, dbArtistsAssertUnofficial

In [ ]:
for modVal in range(100):
    dbAP = dbArtistsPrimary(dbArtistsLastFM())
    dometa = dbAP.parse(modVal, previousDays=2, force=False)
    if dometa:
        dbAP.createArtistMetadata(modVal)
        dbAP.createAlbumMetadata(modVal)

# AlbumOfTheYear

In [ ]:
%load_ext autoreload
%autoreload
from dbArtistsAlbumOfTheYear import dbArtistsAlbumOfTheYear
from dbArtistsParse import dbArtistsPrimary, dbArtistsCredit, dbArtistsExtra
from dbArtistsParse import dbArtistsAssertExtra, dbArtistsUnofficial, dbArtistsAssertUnofficial

In [ ]:
for modVal in range(100):
    dbAP = dbArtistsPrimary(dbArtistsAlbumOfTheYear())
    dometa = dbAP.parse(modVal, previousDays=1, force=True)
    if dometa:
        dbAP.createArtistMetadata(modVal)
        dbAP.createAlbumMetadata(modVal)

# Genius

In [ ]:
%load_ext autoreload
%autoreload
from dbArtistsGenius import dbArtistsGenius
from dbArtistsParse import dbArtistsPrimary, dbArtistsCredit, dbArtistsExtra
from dbArtistsParse import dbArtistsAssertExtra, dbArtistsUnofficial, dbArtistsAssertUnofficial

In [ ]:
for modVal in range(100):
    dbAP = dbArtistsPrimary(dbArtistsGenius())
    dometa = dbAP.parse(modVal, previousDays=1, force=True)
    if dometa:
        dbAP.createArtistMetadata(modVal)
        dbAP.createAlbumMetadata(modVal)

In [ ]:
https://www.kkbox.com/hk/en/artist/FGNJ6VipM2tycJ0F0kas508K-index-1.html

In [ ]:
#dbAP = dbArtistsLastFM()
retval = artistLastFM().getData("/Users/tgadfort/Desktop/Gabo Parisi albums and discography _ Last.fm.html")
retval.show()

In [ ]:
bsdata = getHTML("/Users/tgadfort/Desktop/Artist Search _ Last.fm.html")

In [ ]:
import urllib
def downloadURL(url):
    user_agent = 'Mozilla/5.0 (Windows; U; Windows NT 5.1; en-US; rv:1.9.0.7) Gecko/2009021910 Firefox/3.0.7'
    headers={'User-Agent':user_agent,} 

    print("Now Downloading {0}".format(url))

    request=urllib.request.Request(url,None,headers) #The assembled request
    response = urllib.request.urlopen(request)
    data = response.read() # The data u need

    return data, response.getcode()



In [ ]:
artist = "Genesis"
url = "https://www.albumoftheyear.org/search/?q=S.+Tarabarova"
url = "https://www.albumoftheyear.org/search/?q={0}".format(artist)

In [ ]:
data, code = downloadURL(url)
from ioUtils import getFile, saveFile
saveFile(idata=data, ifile="aoty_{0}.p".format(artist))

In [ ]:
from webUtils import getHTML
bsdata = getHTML("aoty_{0}.p".format(artist))

In [ ]:
for div in bsdata.findAll("div", {"class": "section"}):
    refs = div.findAll("a")
    for ref in refs:
        if ref.find("img") is not None:
            continue
        href = ref.attrs['href']
        name = ref.text
        print(name,href)
        continue
        base = "https://www.albumoftheyear.org"
        url = "{0}{1}?type=all".format(base, href)
        data,code = downloadURL(url)
        saveFile(idata=data, ifile="aoty_Tarabarova.p")        

In [ ]:
bsdata = getHTML("aoty_Tarabarova.p")

In [ ]:
dbAP = dbArtistsAlbumOfTheYear()
dbAP.searchForArtist("Genesis")

In [ ]:
%load_ext autoreload
%autoreload
dbAP = dbArtistsAlbumOfTheYear()
retval = dbAP.artist.getData("/Users/tgadfort/Desktop/Genesis Albums, Songs - Discography - Album of The Year.htm")
retval.show()

In [ ]:
retval.show()

In [ ]:
bsdata

In [ ]:

{
	"@context":"http://schema.org",
	"@type":"MusicGroup",
	"name":"Tarabarova"
})

In [ ]:
dbAP = dbArtistsLastFM()
files = findExt("/Volumes/Piggy/Discog/artists-lastfm/0", ".p")
errs = []
for i,ifile in enumerate(files):
    retval = dbAP.artist.getData(ifile)
    print(retval.ID.ID,'\t',getBaseFilename(ifile),'\t',retval.meta.url)
    if retval.ID.ID != getBaseFilename(ifile):
        errs.append(ifile)

In [ ]:
from musicDBMap import musicDBMap
mdbmap = musicDBMap("Music", init=False, copy=False)

In [ ]:
for i,(primaryKey,artistName) in enumerate(mdbmap.getArtists().items()):
    if artistName == "Fuck":
        print(i)
        break

In [ ]:
from time import sleep
for i,(primaryKey,artistName) in enumerate(mdbmap.getArtists().items()):
    if i < 4567:
        continue
    print("i = {0}".format(i))
    dbAP = dbArtistsLastFM()
    dbAP.searchForArtist(artistName,force=True,maxArtists=2,debug=True)
    sleep(1.5)

In [ ]:
Benny Jamz: https://www.deezer.com/us/artist/5395257/top_track

In [ ]:
https://www.deezer.com/search/Benny%20Jamz

In [ ]:
ifile="/Volumes/Piggy/Discog/artists-lastfm/10/23507355810.p"
retval = artistLastFM().getData(ifile)
retval.show()

In [ ]:
ref="https://www.last.fm/music/Merche%2BCorisco/+albums?order=release_date"
util = utilsLastFM()
print(util.getArtistID(ref))
ref=ref.replace("%2B","+")
print(util.getArtistID(ref))

In [ ]:
ifile="/Volumes/Piggy/Discog/artists-lastfm/13137397800.p"
retval = artistLastFM().getData(ifile)
retval.show()

In [ ]:
bsdata = getHTML("/Users/tgadfort/Desktop/Michael Bublé music, videos, stats, and photos _ Last.fm.html")
alm = artistLastFM().getData("/Users/tgadfort/Desktop/Michael Bublé music, videos, stats, and photos _ Last.fm.html")
alm.show()

In [ ]:
for modVal in range(100):
    dbAP = dbArtistsPrimary(dbArtistsLastFM())
    dbAP.createArtistMetadata(modVal)
    dbAP.createAlbumMetadata(modVal)

In [ ]:
from ioUtils import getFile
data = getFile("/Users/tgadfort/Music/Discog/db-lastfm/MasterSlimArtistDB.p")

In [ ]:
dataRef = getFile("/Users/tgadfort/Music/Discog/db-lastfm/ArtistIDToName.p")

In [ ]:
dataAlbs = getFile("/Users/tgadfort/Music/Discog/db-lastfm/ArtistIDToAlbumRefs.p")

In [ ]:
from pandas import Series, DataFrame
dataRef['64101467014']

In [ ]:
dataAlbs['64101467014']

In [ ]:
data[data.Name == "Dave Matthews Band"] # == '64101467014'

In [ ]:
#data[data.Name.str.contains("Dave Matthews Band")]

In [ ]:
https://www.last.fm/music/Michael%2BBubl%25C3%25A9
https://www.last.fm/music/Michael%2BBubl%25C3%25A9/+albums?order=release_date

In [ ]:
from webUtils import getHTML
data = open("/Users/tgadfort/Desktop/Search | Last.fm.html", 'rb')
print(data)
artist = "Michael Bublé"
#artist = "Dave Matthews Band"
dbAP = dbArtistsLastFM()
dbAP.searchForArtist(artist)
#dbAP.parseSearchArtist(artist=artist, data=data)


# 1 / 27 	: 36599471936 	 https://www.last.fm/music/Michael%25252BBubl%25C3%25A9/+albums?order=release_date
# 2 / 27 	: 8020065709 	 https://www.last.fm/music/Michael%2BBubl%25C3%25A9/+albums?order=release_date
#                                                       Michael%2BBubl%25C3%25A9

In [ ]:
from searchUtils import findExt
from fsUtils import isFile, moveFile
from fileUtils import getBaseFilename
results = {}
for i in range(19,100):
    files = findExt("/Volumes/Piggy/Discog/artists-lastfm/{0}".format(i), ".p")
    print(i,'\t',len(files))
    alm = artistLastFM()
    for j,ifile in enumerate(files):
        if j % 150 == 0:
            print("   {0}/{1}".format(j,len(files)))
        iFileID = getBaseFilename(ifile)
        retval = alm.getData(ifile)
        artistID = retval.ID.ID
        artistName = retval.artist.name
        artistURL = retval.url.url
        results[artistID] = [artistName,artistURL]
        #print(ifile,'\t',iFileID,'\t',artistID,'\t',artistName)
        if iFileID != artistID:
            srcFile="{0}".format(ifile)
            dstFile="/Volumes/Piggy/Discog/artists-lastfm/{0}/{1}.p".format(str(int(artistID)%100), artistID)
            if not isFile(dstFile):
                moveFile(src=srcFile, dst=dstFile, debug=True)
                continue
            print(ifile)
            retval.show()
            print(srcFile)
            print(dstFile)
            1/0
            break

In [ ]:
!rm /Volumes/Piggy/Discog/artists-lastfm/19/98062006719.p

In [ ]:
alm.getData("/Volumes/Piggy/Discog/artists-lastfm/19/98062006719.p").show()
alm.getData("/Volumes/Piggy/Discog/artists-lastfm/39/12498566839.p").show()

# RockCorner

In [ ]:
%load_ext autoreload
%autoreload
from dbArtistsRockCorner import dbArtistsRockCorner
from dbArtistsParse import dbArtistsPrimary, dbArtistsCredit, dbArtistsExtra
from dbArtistsParse import dbArtistsAssertExtra, dbArtistsUnofficial, dbArtistsAssertUnofficial

In [ ]:
for modVal in range(100):
    dbAP = dbArtistsPrimary(dbArtistsRockCorner())
    dbAP.parse(modVal, force=True)

# Rate Your Music

In [ ]:
ifile="/Volumes/Piggy/Discog/ar"

In [ ]:
url="https://www.last.fm/music/16+Deluxe"
url="16+Deluxe"
url="Michael Bublé"
print(quote(quote(url, encoding='utf-8')))
print(utils.getArtistID(url, debug=True))

In [ ]:
"https://www.last.fm/music/Michael%2BBubl%25C3%25A9"

In [ ]:
url="Michael%Bublé"
print(quote(url, encoding='utf-8'))
print(utils.getArtistID(url, debug=True))

In [ ]:
https://www.last.fm/music/Michael%25%2BBubl%25C3%25A9/+albums?order=release_date

In [ ]:
quote(url)
utils.getArtistID("https://www.last.fm/music/Michael%2BBubl%25C3%25A9/+albums?order=release_date", debug=True)

In [ ]:
#help(quote)

In [ ]:

#url="https://www.last.fm/music/Michael+Bublé"
url="https://www.last.fm/music/Dave+Matthews+Band"
url="https://www.last.fm/music/16+Deluxe"                               # 53342847883
from urllib.parse import quote
print(url)
print(quote(url))
#url="https://www.last.fm/music/16+Deluxe/+albums?order=release_date"   # 53342847883
url="https://www.last.fm/music/16%2BDeluxe/+albums?order=release_date" # 99404621521
#url="https://www.last.fm/music/Michael%2BBubl%25C3%25A9/+albums?order=release_date"
#url="https://www.last.fm/music/Michael%25252BBubl%25C3%25A9/+albums?order=release_date"
#url="https://www.last.fm/music/Michael%2BBubl%25C3%25A9/+albums?order=release_date"
from unicodedata import normalize
#‘NFC’, ‘NFKC’, ‘NFD’, and ‘NFKD’.
print(url)
print(normalize("NFC", url))
print(normalize("NFKC", url))
print(normalize("NFD", url))
print(normalize("NFKD", url))
print(utils.getArtistID(url))
print(utils.getArtistID(normalize("NFC", url)))
print(utils.getArtistID(normalize("NFKC", url)))
print(utils.getArtistID(normalize("NFD", url)))
print(utils.getArtistID(normalize("NFKD", url)))

#ifile="/Volumes/Piggy/Discog/artists-lastfm/47/92143197647.p"
#from artistLastFM import artistLastFM
#alm = artistLastFM()
#retval = alm.getData(ifile)
#retval.show()

In [ ]:
#data = getHTML("/Volumes/Piggy/Discog/artists-lastfm/47/85771689147.p")
#data

In [ ]:

ifile="/Volumes/Piggy/Discog/artists-lastfm/50/86362214250.p"
from artistLastFM import artistLastFM
alm = artistLastFM()
1/0
try:
    retval = alm.getData(ifile)
    retval.show()
except:
    print("HI")

In [ ]:



from dbUtils import utilsLastFM
utils = utilsLastFM()
print(utils.getArtistID(url))
ifile = "/Volumes/Piggy/Discog/artists-lastfm/89/83689536289.p"

url = "https://www.last.fm/music/Michael+Bublé"
print(utils.getArtistID(url))

In [ ]:
ifile="/Volumes/Piggy/Discog/artists-lastfm/0/96441297300.p"

In [ ]:
from ioUtils import saveFile, getsFile
from artistLastFM import artistLastFM
alm = artistLastFM()
retval = alm.getData(ifile)
retval.show()

In [ ]:
from dbArtistsDiscogs import dbArtistsDiscogs
for modVal in range(1):
    dbAP = dbArtistsPrimary(dbArtistsDiscogs())
    dbAP.parse(modVal, previousDays=1000)

In [ ]:
dbAP.parse(modVal, previousDays=1000)

# RockCorner

In [ ]:
from dbArtistsRockCorner import dbArtistsRockCorner
from dbArtistsParse import dbArtistsPrimary
dbAP = dbArtistsPrimary(dbArtistsRockCorner())
dbAP.parse(69, previousDays=50)

In [ ]:
from dbArtistsRockCorner import dbArtistsRockCorner
from dbArtistsParse import dbArtistsPrimary
dbAP = dbArtistsPrimary(dbArtistsRockCorner())

In [ ]:
from dbArtistsAllMusic import dbArtistsAllMusic
dbAP = dbArtistsCredit(dbArtistsAllMusic())
dbAP.parse(69, previousDays=50)

In [ ]:
from dbArtistsAllMusic import dbArtistsAllMusic
dbAP = dbArtistsExtra(dbArtistsAllMusic())
dbAP.getArtistFiles(69, force=True)

In [ ]:

        saveIt = 0
        for j,ifile in enumerate(newFiles):
            if force is True:
                if j % 100 == 0:
                    print("\tProcessed {0}/{1} files.".format(j,len(newFiles)))
            artistID = getBaseFilename(ifile)
            isKnown  = dbdata.get(artistID)
            info   = artistInfo.getData(ifile)

            if info.ID.ID != artistID:
                print("File: {0}".format(ifile))
                print(info.show())
                1/0

                # Check Profile
                try:
                    artistName = info.profile.search
                    if artistName is not None:
                        self.creditToDownload[artistID] = [artistName,self.getArtistSavename(artistID, credit=True)]
                except:
                    pass

                if debug is False:
                    continue
                print("ID From Name: {0}".format(artistID))
                print("ID From File: {0}".format(info.ID.ID))

                print("File: {0}".format(ifile))
                print("Info: {0}".format(info.url.get()))
                continue
                #1/0

            saveIt += 1
            dbdata[artistID] = info

               
        forceSave = False
        if saveIt > 0 and doExtra is True:
            print("\tCalling Extra Parsing")
            dbdata = self.parseArtistModValExtraFiles(modVal, dbdata=dbdata, force=force, debug=debug)
            forceSave = True
            saveIt = len(dbdata)
            
         
        if len(self.creditToDownload) > 0:
            print("Found {0} artists that need new downloads.".format(len(self.creditToDownload)))
        

        if saveIt > 0 or forceSave is True:
            savename = setFile(artistDBDir, "{0}-DB.p".format(modVal))     
            print("Saving {0} new artist IDs to {1}".format(saveIt, savename))
            dbNumAlbums = sum([self.getArtistNumAlbums(artistData) for artistData in dbdata.values()])
            print("Saving {0} total artist media".format(dbNumAlbums))
            saveFile(idata=dbdata, ifile=savename)
            
            self.createArtistModValMetadata(modVal=modVal, db=dbdata, debug=debug)
            self.createArtistAlbumModValMetadata(modVal=modVal, db=dbdata, debug=debug)
            
        return saveIt
    
        
        
        
                for j,ifile in enumerate(newFiles):
            if force is True:
                if j % 100 == 0:
                    print("\tProcessed {0}/{1} files.".format(j,len(newFiles)))
            artistID = getBaseFilename(ifile)
            isKnown  = dbdata.get(artistID)
            info   = artistInfo.getData(ifile)

    

In [ ]:

        
        
                    

        
def dbArtistsModValBase:
    def __init__(self, debug=False):
        self.debug = debug
        
    def findFiles(self):

        
    def 
            if info.ID.ID != artistID:
                print("File: {0}".format(ifile))
                print(info.show())
                1/0

                # Check Profile
                try:
                    artistName = info.profile.search
                    if artistName is not None:
                        self.creditToDownload[artistID] = [artistName,self.getArtistSavename(artistID, credit=True)]
                except:
                    pass

                if debug is False:
                    continue
                print("ID From Name: {0}".format(artistID))
                print("ID From File: {0}".format(info.ID.ID))

                print("File: {0}".format(ifile))
                print("Info: {0}".format(info.url.get()))
                continue
                #1/0

            saveIt += 1
            dbdata[artistID] = info


In [ ]:
url="https://kworb.net/itunes/extended.html"
savename="kworb.p"
data, code = downloadURL(url)
saveFile(idata=data, ifile=savename)

In [ ]:
bsdata = getHTML(savename)

In [ ]:
bsdata

In [ ]:
url="https://kworb.net/itunes/artist/theweeknd.html"
savename="kworb_weeknd.p"
data, code = downloadURL(url)
saveFile(idata=data, ifile=savename)

In [ ]:
bsdata = getHTML(savename)

In [ ]:
from artistDBBase import artistDBBase, artistDBDataClass
from artistDBBase import artistDBNameClass, artistDBMetaClass, artistDBIDClass, artistDBURLClass, artistDBURLInfo, artistDBPageClass
from artistDBBase import artistDBProfileClass, artistDBMediaClass, artistDBMediaAlbumClass
from artistDBBase import artistDBMediaDataClass, artistDBMediaCountsClass
from strUtils import fixName
#from dbUtils import utilsKWorb
from webUtils import removeTag


class artistKWorb(artistDBBase):
    def __init__(self, debug=False):
        super().__init__(debug)
        #self.dbUtils = utilsKWorb()
        
        
    ##############################################################################################################################
    ## Parse Data
    ##############################################################################################################################
    def getData(self, inputdata):
        self.getDataBase(inputdata)
        self.checkData()
        
        artist      = self.getName()
        meta        = self.getMeta()
        url         = self.getURL()
        ID          = self.getID(artist)
        pages       = self.getPages()
        profile     = self.getProfile()
        media       = self.getMedia(artist, url)
        mediaCounts = self.getMediaCounts(media)
        
        err = [artist.err, meta.err, url.err, ID.err, pages.err, profile.err, mediaCounts.err, media.err]
        
        adc = artistDBDataClass(artist=artist, meta=meta, url=url, ID=ID, pages=pages, profile=profile, mediaCounts=mediaCounts, media=media, err=err)
        
        return adc
    
    

    ##############################################################################################################################
    ## Artist Name
    ##############################################################################################################################
    def getName(self):
        #<div class="subcontainer"><span class="pagetitle">The Weeknd | <a href="/itunes/">Back to index</a> | <a href="/youtube/artist/theweeknd.html">YouTube stats</a> | 02/06 19:35 EST</span></div>
        span = self.bsdata.find("span", {"class": "pagetitle"})
        if artistData is None:
            anc = artistDBNameClass(err="NoSpan")
            return anc
        
        text = span.text.split(" | ")
        artistName = text[0]
        anc = artistDBNameClass(name=artistName, err=None)
        return anc
    
    

    ##############################################################################################################################
    ## Meta Information
    ##############################################################################################################################
    def getMeta(self):
        metatitle = self.bsdata.find("meta", {"property": "og:title"})
        metaurl   = self.bsdata.find("meta", {"property": "og:url"})

        title = None
        if metatitle is not None:
            title = metatitle.attrs['content']

        url = None
        if metatitle is not None:
            url = metaurl.attrs['content']

        amc = artistDBMetaClass(title=title, url=url)
        return amc
        

    ##############################################################################################################################
    ## Artist URL
    ##############################################################################################################################
    def getURL(self):
        #<div class="subcontainer"><span class="pagetitle">The Weeknd | <a href="/itunes/">Back to index</a> | <a href="/youtube/artist/theweeknd.html">YouTube stats</a> | 02/06 19:35 EST</span></div>
        span = self.bsdata.find("span", {"class": "pagetitle"})
        if artistData is None:
            anc = artistDBNameClass(err="NoSpan")
            return anc
        
        ref
        text = span.text.split(" | ")
        artistName = text[0]
        anc = artistDBNameClass(name=artistName, err=None)
        return anc
        
        artistData = self.bsdata.find("meta", {"property": "og:url"})
        if artistData is None:
            auc = artistDBURLClass(err=True)
            return auc
        
        url = artistData.attrs["content"]
        if url.find("/artist/") == -1:
            url = None
            auc = artistDBURLClass(url=url, err="NoArtist")
        else:
            auc = artistDBURLClass(url=url)

        return auc

    

    ##############################################################################################################################
    ## Artist ID
    ##############################################################################################################################
    def getID(self, artist):
        #discID = self.dbUtils.getArtistID(artist.name)
        discID = None
        aic = artistDBIDClass(ID=discID)
        return aic


    
    ##############################################################################################################################
    ## Artist Pages
    ##############################################################################################################################
    def getPages(self):
        apc   = artistDBPageClass(ppp=1, tot=1, redo=False, more=False)
        return apc
    
    

    ##############################################################################################################################
    ## Artist Variations
    ##############################################################################################################################
    def getProfile(self):
        data    = {}
        apc = artistDBProfileClass(profile=data.get("Formed"), aliases=data.get("Aliases"),
                                 members=data.get("Members"), groups=data.get("In Groups"),
                                 sites=data.get("Sites"), variations=data.get("Variations"))
        return apc

    
    
    ##############################################################################################################################
    ## Artist Media
    ##############################################################################################################################
    def getMediaAlbum(self, td):
        amac = artistDBMediaAlbumClass()
        for span in td.findAll("span"):
            attrs = span.attrs
            if attrs.get("class"):
                if 'format' in attrs["class"]:
                    albumformat = span.text
                    albumformat = albumformat.replace("(", "")
                    albumformat = albumformat.replace(")", "")
                    amac.format = albumformat
                    continue
            span.replaceWith("")

        ref = td.find("a")
        if ref:
            amac.url   = ref.attrs['href']
            amac.album = ref.text
        else:
            amac.err = "NoText"

        return amac
    
    
    def getMedia(self, artist):
        amc  = artistDBMediaClass()
        
        mediaType = "Albums"
        amc.media[mediaType] = []

        for table in self.bsdata.findAll("table"):
            trs = table.findAll("tr")
            for itr,tr in enumerate(trs):
                tds = tr.findAll("td")
                for itd,td in enumerate(tds):
                    div=td.find("div", {"class": "wrap"})
                    if div is not None:
                        name = div.text
                        if name.startswith("Album: "):
                            mediaType = "Album"
                            title = name[7:]
                        else:
                            mediaType = "Single"
                            title = name
                        if media.get(mediaType) is None:
                            media[mediaType] = []

                        amdc = artistDBMediaDataClass(album=title, url=None, aclass=None, aformat=None, artist=artist, code=None, year=None)
                        if amc.media.get(mediaType) is None:
                            amc.media[mediaType] = []
                        amc.media[mediaType].append(amdc)                            

        return amc
    
    

    ##############################################################################################################################
    ## Artist Media Counts
    ##############################################################################################################################
    def getMediaCounts(self, media):
        
        amcc = artistDBMediaCountsClass()
        
        credittype = "Releases"
        if amcc.counts.get(credittype) == None:
            amcc.counts[credittype] = {}
        for creditsubtype in media.media.keys():
            amcc.counts[credittype][creditsubtype] = int(len(media.media[creditsubtype]))
            
        return amcc

In [ ]:
ak = artistKWorb()
retval = ak.getData("kworb_weeknd.p")
retval.show()

In [ ]:
bsdata

In [ ]:
dbArtists = dbArtistsAllMusic()
        dbdata[dbArtists.db] = [dbArtists.disc, dbArtists, dbArtists.artist, dbArtists.dutils]


In [ ]:
from fsUtils import setDir, isDir, mkDir, mkSubDir, setFile, isFile, setSubFile
from ioUtils import saveFile, getFile
from fileUtils import getBaseFilename
from searchUtils import findExt
import urllib
from datetime import datetime, timedelta




class dbArtistsBase():
    def __init__(self, db, disc, artist, dutils, basedir=None, debug=False):
            
        #######################
        ## General Imports
        #######################
        self.db     = db
        self.disc   = disc
        self.name   = "artists"
        self.artist = artist
        self.dutils = dutils
        self.sleeptime=2
        self.debug  = debug
        
        self.getArtistsDir       = self.disc.getArtistsDir
        self.getArtistsDBDir     = self.disc.getArtistsDBDir
        self.getDiscogDBDir      = self.disc.getDiscogDBDir
        
        self.prevSearches        = {}
        
        self.modVal = self.disc.getMaxModVal
        
        self.creditToDownload = {}
        
        self.artistIDtoRefData = None
            
    
    ###############################################################################
    # Artist Data
    ###############################################################################
    def getData(self, ifile):
        info = self.artist.getData(ifile)
        return info
    
    def getFileData(self, artistID):
        ifile = self.getArtistSavename(artistID, 1)
        info  = self.getData(ifile)
        return info
    
    
    ###############################################################################
    # ModVals
    ###############################################################################
    def getModVals(self):
        return self.disc.getModValList()
        #return [str(x) for x in range(self.disc.getMaxModVal)]
    
    def getModValDirs(self):
        modVals = self.getModVals()
        retval  = [setDir(self.getArtistsDir(), str(modVal)) for modVal in modVals]
        return retval
    
    
    ###############################################################################
    # Download Information
    ###############################################################################
    def getArtistURL(self, artistRef, page=1, credit=False, unofficial=False):
        raise ValueError("Override getArtistURL")
    
    def getArtistSavename(self, discID, page=1, credit=False, unofficial=False):
        artistDir = self.disc.getArtistsDir()
        modValue  = self.dutils.getDiscIDHashMod(discID=discID, modval=self.disc.getMaxModVal())
        if modValue is not None:
            outdir    = mkSubDir(artistDir, str(modValue))
            if isinstance(page, int) and page > 1:
                outdir = mkSubDir(outdir, "extra")
                savename  = setFile(outdir, discID+"-{0}.p".format(page))
            elif credit is True:
                outdir = mkSubDir(outdir, "credit")
                savename  = setFile(outdir, discID+".p")
            elif unofficial is True:
                outdir = mkSubDir(outdir, "unofficial")
                savename  = setFile(outdir, discID+".p")
            else:
                savename  = setFile(outdir, discID+".p")
                
            return savename
        return None
    
    
    def downloadURL(self, url):
        user_agent = 'Mozilla/5.0 (Windows; U; Windows NT 5.1; en-US; rv:1.9.0.7) Gecko/2009021910 Firefox/3.0.7'
        headers={'User-Agent':user_agent,} 

        if self.debug:
            print("Now Downloading {0}".format(url))

        request=urllib.request.Request(url,None,headers) #The assembled request
        response = urllib.request.urlopen(request)
        data = response.read() # The data u need
        
        return data, response.getcode()


    def downloadArtistFromID(self, artistID, artistRef, force=False):
        if self.debug:
            print("Downloading Artist Data for ID [{0}] and Ref [{1}]".format(artistID, artistRef))
        url = self.getArtistURL(artistRef)
        savename = self.getArtistSavename(artistID)
        retval = self.downloadArtistURL(url, savename, force=force)
        

    ###############################################################################
    # Download Functions
    ###############################################################################
    def downloadArtistURL(self, url, savename, force=False, sleeptime=2):
        if isFile(savename):
            if self.debug:
                print("{0} exists.".format(savename))
            if force is False:
                return False
            else:
                print("Downloading again.")
                  
        ## Download data
        data, response = self.downloadURL(url)
        if response != 200:
            print("Error downloading {0}".format(url))
            return False
            
        print("Saving {0} (force={1})".format(savename, force))
        saveFile(idata=data, ifile=savename)
        print("Done. Sleeping for {0} seconds".format(sleeptime))
        sleep(sleeptime)
        
        if isFile(savename):
            return True
        else:
            return False
    

    def downloadArtistCreditURL(self, artistData, debug=False, force=False):
        artistRef = artistData.url.url
        artistID  = artistData.ID.ID
        print("Downloading credit URL for ArtistID {0}".format(artistID))

        url      = self.getArtistURL(artistRef, credtit=True)
        savename = self.getArtistSavename(artistID, credit=True)
        if not isFile(savename) or force is True:
            retval = self.downloadArtistURL(url=url, savename=savename, force=force)
            return retval
        return False
    

    def downloadArtistUnofficialURL(self, artistData, debug=False, force=False):
        artistRef = artistData.url.url
        artistID  = artistData.ID.ID
        print("Downloading credit URL for ArtistID {0}".format(artistID))

        url      = self.getArtistURL(artistRef, unofficial=True)
        savename = self.getArtistSavename(artistID, unofficial=True)
        if not isFile(savename) or force is True:
            retval = self.downloadArtistURL(url=url, savename=savename, force=force)
            return retval
        return False
            
    

    def downloadArtistExtraURL(self, artistData, debug=False, force=False):
        newPages = 0
        pages = artistData.pages
        if pages.more is True:
            npages = pages.pages
            artistRef = artistData.url.url
            artistID  = artistData.ID.ID
            print("Downloading an additional {0} URLs for ArtistID {1}".format(npages-1, artistID))

            for p in range(2, npages+1):
                url      = self.getArtistURL(artistRef, p)
                savename = self.getArtistSavename(artistID, p)
                if not isFile(savename) or force is True:
                    self.downloadArtistURL(url=url, savename=savename, force=True, debug=True)
                    newPages += 1
                    
        return newPages

            
    ################################################################################
    # Search For Artist
    ################################################################################
    def searchForArtist(self, artist):
        raise ValueError("Override searchForArtist")
        

    ################################################################################
    # Parse Artist Data
    ################################################################################
    def getArtistNumAlbums(self, artistData):
        numAlbums = sum([len(x) for x in artistData.media.media.values()])
        return numAlbums
    
    
    def parseArtistFile(ifile):
        bsdata     = getHTML(get(ifile))
        artistData = self.parse(bsdata) 
        return artistData
    
    
    
    def parseArtistModValCreditFiles(self, modVal, dbdata=None, debug=False, force=False):
        print("\t","="*100)
        print("\t","Parsing Artist Credit Files For ModVal {0}".format(modVal))
        artistInfo = self.artist

        artistDir = self.disc.getArtistsDir()
        maxModVal = self.disc.getMaxModVal()
                    
        artistDBDir = self.disc.getArtistsDBDir()        
        
        dirVal = setDir(artistDir, str(modVal))
        dirVal = setDir(dirVal, "credit")
        files  = findExt(dirVal, ext='.p')
        
        if len(files) == 0:
            return dbdata
        print("\t","  Found {0} credit files for ModVal {1}".format(len(files), modVal))

        dbname = setFile(artistDBDir, "{0}-DB.p".format(modVal))
        retdbdata = False

        if dbdata is None:
            print("\t","  Loaded ", end="")
            dbdata = getFile(dbname, version=3)
            print("\t","{0} artist IDs.".format(len(dbdata)))
        else:
            retdbdata = True

        saveIt = 0
        
        nArtistMedia = {}
        print("\t","{0} artist IDs.".format(len(dbdata)))
        
        for j,ifile in enumerate(files):
            if force is True:
                if j % 500 == 0:
                    print("\t","\tProcessed {0}/{1} files.".format(j,len(files)))
            if debug:
                print("\t","{0}/{1} -- {2}.".format(j,len(files),ifile))
            
            info     = artistInfo.getData(ifile)
            artistID = info.ID.ID
            
            #currentMedia = sum([len(x) for x in dbdata[artistID].media.media.values()])
            #print(artistID,'\t',sum([len(x) for x in dbdata[artistID].media.media.values()]),end="\t")

            currentKeys = []
            if dbdata.get(artistID) is not None:
                currentKeys = list(dbdata[artistID].media.media.keys())
            else:
                dbdata[artistID] = info
                saveIt += 1
                continue
            
            keys = list(set(list(info.media.media.keys()) + currentKeys))
            for k in keys:
                v = info.media.media.get(k)
                if v is None:
                    continue
                iVal  = {v2.code: v2 for v2 in v}
                dVal  = dbdata[artistID].media.media.get(k)
                if dVal is None:
                    Tretval = iVal
                    saveIt += len(iVal)
                else:
                    Tretval = {v2.code: v2 for v2 in dVal}
                    Tretval.update(iVal)
                    saveIt += len(iVal)
                dbdata[artistID].media.media[k] = list(Tretval.values())
                
            if debug:
                print("\t","File:",j," \tArtist:",artistID,'-->',currentMedia,'to',sum([len(x) for x in dbdata[artistID].media.media.values()]))

                
        if retdbdata is True:
            return dbdata
        #if saveAll is False:
        #    return saveIt
                
                
        if saveIt > 0:
            savename = setFile(artistDBDir, "{0}-DB.p".format(modVal))
            print("\t","Saving {0} artists to {1}".format(len(dbdata), savename))
            print("\t","Saving {0} new (credit) artist media to {1}".format(saveIt, savename))
            dbNumAlbums = sum([self.getArtistNumAlbums(artistData) for artistData in dbdata.values()])
            print("\t","Saving {0} total (credit) artist media".format(dbNumAlbums))
            saveFile(idata=dbdata, ifile=savename)
            
            self.createArtistModValMetadata(modVal=modVal, db=dbdata, debug=debug)
            self.createArtistAlbumModValMetadata(modVal=modVal, db=dbdata, debug=debug)
            
        return saveIt

    
    
    ###########################################################################################################################
    ##
    ## Extra ModVal Files
    ##
    ###########################################################################################################################
    def getArtistModValExtraFiles(self, modVal, previousDays=5, force=False):
        artistDir = self.disc.getArtistsDir()
        maxModVal = self.disc.getMaxModVal()
                    
        artistDBDir = self.disc.getArtistsDBDir()        
        
        dirVal = setDir(artistDir, str(modVal))
        dirVal = setDir(dirVal, "extra")
        files  = findExt(dirVal, ext='.p')
        dbname = setFile(artistDBDir, "{0}-DB.p".format(modVal))
        
        now    = datetime.now()
        newFiles = None
        if lastModified is None:
            newFiles = files
            print("  ===> Parsing all {0} extra files for modval {1}".format(len(newFiles), modVal))
        else:
            numFiles = [ifile for ifile in files if (now-datetime.fromtimestamp(path.getmtime(ifile))).days < previousDays]
            print("  ===> Found new {0} extra files (< {1} days) to parse for modval {2}".format(len(newFiles), previousDays, modVal))
        return newFiles
    
    
    
    def parseArtistModValExtraFiles(self, modVal, dbdata=None, debug=False, force=False):
        print("\t","="*100)
        print("\t","Parsing Artist Extra Files For ModVal {0}".format(modVal))
        artistInfo = self.artist

        newFiles = self.getArtistModValExtraFiles(modVal, previousDays=previousDays, force=force)
        
        if len(newFiles) == 0:
            return dbdata
        print("\t","  Found {0} extra files for ModVal {1}".format(len(newFiles), modVal))

        dbname = setFile(artistDBDir, "{0}-DB.p".format(modVal))
        retdbdata = False

        if force is False:
            if dbdata is None:
                print("\t","  Loaded ", end="")
                dbdata = getFile(dbname, version=3)
                print("\t","{0} artist IDs.".format(len(dbdata)))
            else:
                retdbdata = True
        else:
            print("\t","Forcing Reloads of ModVal={0}".format(modVal))
            print("\t","  Processing {0} files.".format(len(files)))
            dbdata = {}

        saveIt = 0
        
        nArtistMedia = {}
        print("\t","{0} artist IDs.".format(len(dbdata)))

        
        
        for j,ifile in enumerate(newFiles):
            if force is True:
                if j % 50 == 0:
                    print("\t","\tProcessed {0}/{1} files.".format(j,len(newFiles)))
            if debug:
                print("\t","{0}/{1} -- {2}.".format(j,len(newFiles),ifile))
            
            info     = artistInfo.getData(ifile)
            artistID = info.ID.ID
            
            currentMedia = sum([len(x) for x in dbdata[artistID].media.media.values()])
            #print(artistID,'\t',sum([len(x) for x in dbdata[artistID].media.media.values()]),end="\t")

            keys = list(set(list(info.media.media.keys()) + list(dbdata[artistID].media.media.keys())))
            for k in keys:
                v = info.media.media.get(k)
                if v is None:
                    continue
                iVal  = {v2.code: v2 for v2 in v}
                dVal  = dbdata[artistID].media.media.get(k)
                if dVal is None:
                    Tretval = iVal
                    saveIt += len(iVal)
                else:
                    Tretval = {v2.code: v2 for v2 in dVal}
                    Tretval.update(iVal)
                    saveIt += len(iVal)
                dbdata[artistID].media.media[k] = list(Tretval.values())
                
            if debug:
                print("\t","File:",j," \tArtist:",artistID,'-->',currentMedia,'to',sum([len(x) for x in dbdata[artistID].media.media.values()]))

                
        if retdbdata is True:
            return dbdata
        #if saveAll is False:
        #    return saveIt
                
                
        if saveIt > 0:
            savename = setFile(artistDBDir, "{0}-DB.p".format(modVal))     
            print("\t","Saving {0} new (extra) artist media to {1}".format(saveIt, savename))
            dbNumAlbums = sum([self.getArtistNumAlbums(artistData) for artistData in dbdata.values()])
            print("\t","Saving {0} total (extra) artist media".format(dbNumAlbums))
            saveFile(idata=dbdata, ifile=savename)
            
            self.createArtistModValMetadata(modVal=modVal, db=dbdata, debug=debug)
            self.createArtistAlbumModValMetadata(modVal=modVal, db=dbdata, debug=debug)
            
        return saveIt

    
    
    ###########################################################################################################################
    ##
    ## Regular ModVal Files
    ##
    ###########################################################################################################################
    def getArtistModValFiles(self, modVal, previousDays=5, force=False):
        artistDir = self.disc.getArtistsDir()
        maxModVal = self.disc.getMaxModVal()
                    
        artistDBDir = self.disc.getArtistsDBDir()        
        
        dirVal = setDir(artistDir, str(modVal))
        files  = findExt(dirVal, ext='.p')
        dbname = setFile(artistDBDir, "{0}-DB.p".format(modVal))
        
        now    = datetime.now()
        if isFile(dbname):
            lastModified = datetime.fromtimestamp(path.getmtime(dbname))
            if force is True:
                lastModified = None
        else:
            lastModified = None

        newFiles = None
        if lastModified is None:
            newFiles = files
            print("  ===> Parsing all {0} files for modval {1}".format(len(newFiles), modVal))
        else:
            numNew    = [ifile for ifile in files if (now-datetime.fromtimestamp(path.getmtime(ifile))).days < previousDays]
            numRecent = [ifile for ifile in files if datetime.fromtimestamp(path.getmtime(ifile)) > lastModified]
            newFiles  = list(set(numNew).union(set(numRecent)))
            print("  ===> Found new {0} files (< {1} days) to parse for modval {2}".format(len(newFiles), previousDays, modVal))
        return newFiles
            
    
    def parseArtistModValFiles(self, modVal, previousDays=5, force=False, debug=False, doExtra=False):        
        print("-"*100)
        print("Parsing Artist Files For ModVal {0}".format(modVal))
        artistInfo = self.artist

        artistDir = self.disc.getArtistsDir()
        maxModVal = self.disc.getMaxModVal()
                    
        artistDBDir = self.disc.getArtistsDBDir()        
        
        dirVal = setDir(artistDir, str(modVal))
        dbname = setFile(artistDBDir, "{0}-DB.p".format(modVal))

        
        newFiles = self.getArtistModValFiles(modVal, previousDays=previousDays, force=force)
        if force is False:
            dbdata = getFile(dbname, version=3)
        else:
            print("Forcing Reloads of ModVal={0}".format(modVal))
            print("  Processing {0} files.".format(len(newFiles)))
            dbdata = {}
 
        saveIt = 0
        for j,ifile in enumerate(newFiles):
            if force is True:
                if j % 100 == 0:
                    print("\tProcessed {0}/{1} files.".format(j,len(newFiles)))
            artistID = getBaseFilename(ifile)
            isKnown  = dbdata.get(artistID)
            info   = artistInfo.getData(ifile)

            if info.ID.ID != artistID:
                print("File: {0}".format(ifile))
                print(info.show())
                1/0

                # Check Profile
                try:
                    artistName = info.profile.search
                    if artistName is not None:
                        self.creditToDownload[artistID] = [artistName,self.getArtistSavename(artistID, credit=True)]
                except:
                    pass

                if debug is False:
                    continue
                print("ID From Name: {0}".format(artistID))
                print("ID From File: {0}".format(info.ID.ID))

                print("File: {0}".format(ifile))
                print("Info: {0}".format(info.url.get()))
                continue
                #1/0

            saveIt += 1
            dbdata[artistID] = info

               
        forceSave = False
        if saveIt > 0 and doExtra is True:
            print("\tCalling Extra Parsing")
            dbdata = self.parseArtistModValExtraFiles(modVal, dbdata=dbdata, force=force, debug=debug)
            forceSave = True
            saveIt = len(dbdata)
            
         
        if len(self.creditToDownload) > 0:
            print("Found {0} artists that need new downloads.".format(len(self.creditToDownload)))
        

        if saveIt > 0 or forceSave is True:
            savename = setFile(artistDBDir, "{0}-DB.p".format(modVal))     
            print("Saving {0} new artist IDs to {1}".format(saveIt, savename))
            dbNumAlbums = sum([self.getArtistNumAlbums(artistData) for artistData in dbdata.values()])
            print("Saving {0} total artist media".format(dbNumAlbums))
            saveFile(idata=dbdata, ifile=savename)
            
            self.createArtistModValMetadata(modVal=modVal, db=dbdata, debug=debug)
            self.createArtistAlbumModValMetadata(modVal=modVal, db=dbdata, debug=debug)
            
        return saveIt
    

    def parseArtistFiles(self, force=False, debug=False):           
        totalSaves = 0
        maxModVal  = self.disc.getMaxModVal()
        for modVal in range(maxModVal):
            saveIt = self.parseArtistModValFiles(modVal, force=force, debug=debug)
            totalSaves += saveIt
            
        print("Saved {0} new artist IDs".format(totalSaves)) 

    def parseArtistMetadataFiles(self, debug=False):   
        artistDBDir = self.disc.getArtistsDBDir()   
        maxModVal   = self.disc.getMaxModVal()
        for modVal in range(maxModVal):
            savename = setFile(artistDBDir, "{0}-DB.p".format(modVal))     
            dbdata   = getFile(savename)
            self.createArtistModValMetadata(modVal=modVal, db=dbdata, debug=debug)
            self.createArtistAlbumModValMetadata(modVal=modVal, db=dbdata, debug=debug)
                 
        
        
    
    ################################################################################
    # Check ArtistDB Files
    ################################################################################ 
    def rmIDFiles(self, artistID):
        print("Removing files artistID {0}".format(artistID))
        savename = self.getArtistSavename(artistID)
        if isFile(savename):
            files = [savename]
        else:
            files = []
        from glob import glob
        from os.path import join
        from fileUtils import getDirname
        files += glob(join(getDirname(savename), "extra", "{0}-*.p".format(artistID)))
        print("Found {0} files to delete.".format(len(files)))
        from fsUtils import removeFile
        for ifile in files:
            removeFile(ifile)
            print("Removed File {0}".format(ifile))

                
    def rmIDsFromDBs(self, artistIDs, modValue=None):
        modvals = {}
        for artistID in artistIDs:
            modValue  = self.dutils.getDiscIDHashMod(discID=artistID, modval=self.disc.getMaxModVal())
            if modvals.get(modValue) is None:
                modvals[modValue] = []
            modvals[modValue].append(artistID)
            
        for modval in modvals.keys():
            dbdata = self.disc.getArtistsDBModValData(modval)
            for artistID in modvals[modval]:
                try:
                    del dbdata[artistID]
                    print("  Removed ArtistID {0}".format(artistID))
                except:
                    print("  Could not remove ArtistID {0}".format(artistID))
                    
            self.disc.saveArtistsDBModValData(modval, dbdata)
                


    def rmIDFromDB(self, artistID, modValue=None):
        print("Trying to remove data from ArtistID {0}".format(artistID))
        if modValue is None:
            modValue  = self.dutils.getDiscIDHashMod(discID=artistID, modval=self.disc.getMaxModVal())
        artistDBDir = self.disc.getArtistsDBDir()
        dbname  = setFile(artistDBDir, "{0}-DB.p".format(modValue))     
        print("Loading {0}".format(dbname))
        dbdata  = getFile(dbname)
        
        saveVal = False

        if isinstance(artistID, str):
            artistID = [artistID]
        elif not isinstance(artistID, list):
            raise ValueError("Not sure what to do with {0}".format(artistID))
            
        for ID in artistID:
            try:
                del dbdata[ID]
                print("Deleted {0}".format(ID))
                saveVal = True
            except:
                print("Not there...")

            self.rmIDFiles(ID)

        if saveVal:
            print("Saving {0}".format(dbname))
            saveFile(idata=dbdata, ifile=dbname)
        else:
            print("No reason to save {0}".format(dbname))


            
    def assertDBModValUnofficialData(self, modVal, test=False):
        print("assertDBModValUnofficialData({0}, test={1})".format(modVal, test))
        artistDBDir = self.disc.getArtistsDBDir()
        dbdata  = self.disc.getArtistsDBModValData(modVal)
        nerrs   = 0
        
        for artistID,artistData in dbdata.items():
            mediaCounts = artistData.mediaCounts.counts
            if mediaCounts.get("Unofficial") is not None:
                artistRef = artistData.url.url
                url      = self.getArtistURL(artistRef, unofficial=True)
                savename = self.getArtistSavename(artistID, unofficial=True)
                if test is True:
                    print("Download Unofficial: {0} \t".format(artistID),end="")
                if not isFile(savename):
                    if test is True:
                        print("{0} --> {1}".format(url,savename))
                        continue
                    self.downloadArtistURL(url=url, savename=savename, force=False)
                    sleep(2)
                else:
                    print("Known")


            
    def assertDBModValCreditData(self, modVal, test=False):
        print("assertDBModValCreditData({0}, test={1})".format(modVal, test))
        artistDBDir = self.disc.getArtistsDBDir()
        dbdata  = self.disc.getArtistsDBModValData(modVal)
        nerrs   = 0
        
        for artistID,artistData in dbdata.items():
            mediaCounts = artistData.mediaCounts.counts
            if mediaCounts.get("Credits") is not None:
                artistRef = artistData.url.url
                url      = self.getArtistURL(artistRef, credit=True)
                savename = self.getArtistSavename(artistID, credit=True)
                if test is True:
                    print("Download Credit: {0} \t".format(artistID),end="")
                if not isFile(savename):
                    if test is True:
                        print("{0} --> {1}".format(url,savename))
                        continue
                    self.downloadArtistURL(url=url, savename=savename, force=False)
                    sleep(2)
                else:
                    print("Known")
                
    
    def assertDBModValExtraData(self, modVal, test=False):
        print("assertDBModValExtraData({0}, test={1})".format(modVal, test))
        artistDBDir = self.disc.getArtistsDBDir()
        dbdata  = self.disc.getArtistsDBModValData(modVal)
        nerrs   = 0
        
        for artistID,artistData in dbdata.items():
            pages = artistData.pages
            if pages.more is True:
                npages = pages.pages
                artistRef = artistData.url.url
                for p in range(2, npages+1):
                    url      = self.getArtistURL(artistRef, p)
                    savename = self.getArtistSavename(artistID, p)
                    if test is True:
                        print("Download Extra: {0} \t".format(artistID),end="")
                    if not isFile(savename):
                        if test is True:
                            print("{0} --> {1}".format(url,savename))
                            continue
                        self.downloadArtistURL(url=url, savename=savename, force=True, debug=True)
                        sleep(2)
                    else:
                        print("Known")
                        
            
    def assertDBModValData(self, modVal):
        
        artistDBDir = self.disc.getArtistsDBDir()
        dbname  = setFile(artistDBDir, "{0}-DB.p".format(modVal))     
        dbdata  = getFile(dbname)
        nerrs = 0
        
        if self.artistIDtoRefData is None:
            self.artistIDtoRefData = self.disc.getArtistIDToRefData()
        
        dels = []
        for artistID,artistData in dbdata.items():
            pages = artistData.pages
            if pages.redo is True and False:
                artistRef = artistData.url.url
                url       = self.getArtistURL(artistRef, 1)
                savename  = self.getArtistSavename(artistID, 1)
                self.downloadArtistURL(url=url, savename=savename, force=True, debug=True)

            ID = artistData.ID.ID
            if ID != artistID:

                nerrs += 1

                if "-" in artistID:
                    print("Extra file: {0}".format(artistID))
                    continue
                else:
                    dels.append(artistID)
                    
                    rmsavename = self.getArtistSavename(artistID)


                    ## ID = artistID                    
                    refRef      = self.artistIDtoRefData.get(artistID)
                    if refRef is None:
                        raise ValueError("Ref for ID [{0}] is None!".format(artistID))
                    else:
                        print("ArtistRef:",refRef)
                        urlRef         = self.getArtistURL(refRef)
                        savenameArtRef = self.getArtistSavename(artistID)


                    ## ID = info.ID.ID
                    try:
                        info  = self.getFileData(artistID)
                    except:
                        info  = None

                    if info is not None:
                        try:
                            refIDID      = artistIDtoRefData[info.ID.ID]
                        except:
                            refIDID      = info.url.url
                        print("ArtistID: ",refIDID)
                        urlIDID      = self.getArtistURL(refIDID)
                        savenameIDID = self.getArtistSavename(info.ID.ID)
                    else:
                        refIDID      = None
                        urlIDID      = None
                        savenameIDID = None

                        
                    if isFile(rmsavename):
                        removeFile(rmsavename)


                    if isFile(savenameArtRef):
                        removeFile(savenameArtRef)
                        self.downloadArtistURL(url=urlRef, savename=savenameArtRef, force=True, debug=True)

                    if savenameArtRef != savenameIDID:
                        if isFile(savenameIDID):
                            removeFile(savenameIDID)
                            self.downloadArtistURL(url=urlIDID, savename=savenameIDID, force=True, debug=True)


                    #print(rmsavename,'\t',savenameArtID,'\t',savenameIDID)        
        
        print("Found {0} errors with modVal {1}".format(nerrs, modVal))
        
        dbname  = setFile(artistDBDir, "{0}-DB.p".format(modVal))
        print("Found {0} artist IDs in {1}".format(len(dbdata), dbname))
        
        for artistID in dels:
            print("Deleting {0}".format(artistID))
            try:
                del dbdata[artistID]
            except:
                continue
            
        if len(dels) > 0:
            savename = setFile(artistDBDir, "{0}-DB.p".format(modVal))     
            print("Saving {0} artist IDs to {1}".format(len(dbdata), savename))
            saveFile(idata=dbdata, ifile=savename)
        
        
    
    ################################################################################
    # Collect Metadata About Artists (4)
    ################################################################################
    def createArtistModValMetadata(self, modVal, db=None, debug=False):
        if db is None:
            db = self.disc.getArtistsDBModValData(modVal)
    
        artistIDMetadata = {k: [v.artist.name, v.url.url] for k,v in db.items()}
        
        for artistID,artistData in db.items():
            if artistData.profile.variations is not None:
                artistIDMetadata[artistID].append([v2.name for v2 in artistData.profile.variations])
            else:
                artistIDMetadata[artistID].append([artistData.artist.name])
        
        artistDBDir = self.disc.getArtistsDBDir()     
        savename    = setSubFile(artistDBDir, "metadata", "{0}-Metadata.p".format(modVal))
        
        print("Saving {0} new artist IDs name data to {1}".format(len(artistIDMetadata), savename))
        saveFile(idata=artistIDMetadata, ifile=savename)
        
        
    def createArtistAlbumModValMetadata(self, modVal, db=None, debug=False):
        if db is None:
            db = self.disc.getArtistsDBModValData(modVal)
        
        artistIDMetadata = {}
        for artistID,artistData in db.items():
            artistIDMetadata[artistID] = {}
            for mediaName,mediaData in artistData.media.media.items():
                albumURLs  = {mediaValues.code: mediaValues.url for mediaValues in mediaData}
                albumNames = {mediaValues.code: mediaValues.album for mediaValues in mediaData}
                artistIDMetadata[artistID][mediaName] = [albumNames, albumURLs]
        
        artistDBDir = self.disc.getArtistsDBDir()     
        savename    = setSubFile(artistDBDir, "metadata", "{0}-MediaMetadata.p".format(modVal))
        
        print("Saving {0} new artist IDs media data to {1}".format(len(artistIDMetadata), savename))
        saveFile(idata=artistIDMetadata, ifile=savename)